In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import collections

In [ ]:
# service handle
def ser_h():
    path = '/home/rei/reigit/service_request.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    sh=[]
    for i in l:
        sh.append(i.split('0x')[1].split(',')[0])
    sh2=[]
    for i in sh:
        sh2.append(str(int(i,16)))
    s_counter=collections.Counter(sh2)  #how many time did they run?
    service_counter=dict(s_counter)

    # connect handle to service name
    path = '/home/rei/reigit/avpdemo.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    S = l.index('Services:\n')
    E = l.index('Clients:\n')
    service_sn=[]
    for i in l[S+3:E-1]:
        for j in service_counter.keys():
             if j == i.split()[0]:
                 service_sn.append(j+':'+i.split()[4]+':'+str(service_counter.get(str(j)))+'--'+i.split()[2])
    
    S = l.index('Nodes:\n')
    E = l.index('Publishers:\n')
    service_node=[]
    for i in service_sn:
        for j in l[S:E]:
            if i.split('--')[1] in j:
                service_node.append(i+':'+j.split()[4])
    
    return service_counter,service_node
service_counter,service_node=ser_h()
for i in service_node:
    print(i)

In [ ]:
# client handle
def cli_h():
    path = '/home/rei/reigit/client_response.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    ch=[]
    for i in l:
        ch.append(i.split('0x')[1].split(',')[0])
    ch2=[]
    for i in ch:
        ch2.append(str(int(i,16)))
    c_counter=collections.Counter(ch2)  #how many time did they run?
    client_counter=dict(c_counter)

    # connect handle to service name
    path = '/home/rei/reigit/avpdemo.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    S = l.index('Clients:\n')
    E = l.index('Timers:\n')
    client_sn=[]
    for i in l[S+3:E-1]:
        for j in client_counter.keys():
             if j == i.split()[0]:
                 client_sn.append(j+':'+i.split()[4]+':'+str(client_counter.get(str(j)))+'--'+i.split()[2])
        
    S = l.index('Nodes:\n')
    E = l.index('Publishers:\n')
    client_node=[]
    for i in client_sn:
        for j in l[S:E]:
            if i.split('--')[1] in j:
                client_node.append(i+':'+j.split()[4])
    
    return client_counter,client_node
client_counter,client_node=cli_h()
for i in client_node:
    print(i)

In [ ]:
# only calculate the latency of service which execution time exceed 10
def fil():
    service_c=[]
    for i in service_counter.keys():
        if service_counter.get(i) > 10:
            service_c.append(i)
    service_p=[]
    for i in service_c:
        for j in service_node:
            if i == j.split(':')[0]:
                service_p.append(j)

    client_c=[]
    for i in client_counter.keys():
        if client_counter.get(i) > 10:
            client_c.append(i)
    client_p=[]
    for i in client_c:
        for j in client_node:
            if i == j.split(':')[0]:
                client_p.append(j)

    return service_p,client_p
service_p,client_p=fil()
print('service:')
print(service_p)
print('client:')
print(client_p)

In [ ]:
# find service pairs
def pair():
    pairs=[]
    for i in service_p:
        for j in client_p:
            if i.split(':')[1]==j.split(':')[1]:
                ser=hex(int(i.split(':')[0]))
                cli=hex(int(j.split(':')[0]))
                pairs.append(ser+':'+i.split(':')[3]+'--'+cli+':'+j.split(':')[3]+'--'+j.split(':')[1])
    return pairs
print('service:client:service name')
pairs=pair()
print(pairs)

In [ ]:
# request latency
def req(service_handle):
    path = '/home/rei/reigit/service_request.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    request=[]
    for i in l:
        service = i.split(",")[4].split(" = ")[1]
        if (service == service_handle):
            source = i.split(",")[5].split(" = ")[1]
            receive = i.split(",")[6].split(" = ")[1].split()[0]
            req = int(receive) - int(source)
            request.append(req)

    request_us = []
    for i in request:
        request_us.append(i/1000)
    return request_us

In [ ]:
# response latency
def res(client_handle):
    path = '/home/rei/reigit/client_response.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    response=[]
    for i in l:
        client = i.split(",")[4].split(" = ")[1]
        if (client == client_handle):
            source = i.split(",")[5].split(" = ")[1]
            receive = i.split(",")[6].split(" = ")[1].split()[0]
            res = int(receive) - int(source)
            response.append(res)

    response_us = []
    for i in response:
        response_us.append(i/1000)
    return response_us

In [ ]:
# service callback duration
def ser_cb(service_handle):
    path = '/home/rei/reigit/avpdemo.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    
    #service_handle -> callback_object
    S = l.index('Callback objects:\n')
    E = l.index('Callback symbols:\n')   
    cb= [line for line in l[S:E] if service_handle in line]
    callback_object=[]
    for i in cb:
        callback_object.append(i.split()[2])
    #print(callback_object)
    
    #callback_object -> duration
    S = l.index('Callback instances:\n')
    E = l.index('Lifecycle state machines:\n')
    for i in callback_object:
        d= [line for line in l[S:E] if i in line]
    duration=[]
    for i in d:
        duration.append(i.split()[3])
    #print(duration)
    
    cbd_us = []
    for i in duration:
        cbd_us.append(int(i)/1000)
    return cbd_us

In [ ]:
#check the service latency
def check(ser,req,res,cb):
    if max(ser)>max(max(req),max(res),max(cb)) and min(ser)>max(min(req),min(res),min(cb)):
        print("right output!")
    else:
        print("impossible output!")

In [ ]:
# print('service:client:service name')
# print(pairs)
# for i in pairs:
#     request=req(i.split(':')[0].upper().replace('X', 'x'))
#     #graph(request, "request latency")
#     response=res(i.split(':')[1].upper().replace('X', 'x'))
#     #graph(response, "response latency")
#     cbd=ser_cb(str(int(i.split(':')[0],16)))
#     #graph(cbd, "callback duration")
#     service_latency = [request[j]+response[j]+cbd[j] for j in range(min(len(request),len(response),len(cbd)))]
#     #graph(service_latency, "service latency")
#     check(service_latency,request,response,cbd)

In [ ]:
# a=5
# def req_pr(service_handle):
#     path = '/home/rei/reigit/service_request.txt'
#     f = open(path)
#     f.close()
#     with open(path) as f:
#         l = f.readlines()
#     k=0
#     for i in l:
#         service = i.split(",")[4].split(" = ")[1]
#         if (service == service_handle) and k < a:
#             print(i)
#             k+=1

# def res_pr(client_handle):
#     path = '/home/rei/reigit/client_response.txt'
#     f = open(path)
#     f.close()
#     with open(path) as f:
#         l = f.readlines()
#     k=0
#     for i in l:
#         client = i.split(",")[4].split(" = ")[1]
#         if (client == client_handle) and k < a:
#             print(i)
#             k+=1

# def cb_pr(service_handle):
#     path = '/home/rei/reigit/avpdemo.txt'
#     f = open(path)
#     f.close()
#     with open(path) as f:
#         l = f.readlines()
    
#     #service_handle -> callback_object
#     S = l.index('Callback objects:\n')
#     E = l.index('Callback symbols:\n')   
#     cb= [line for line in l[S:E] if service_handle in line]
#     callback_object=[]
#     for i in cb:
#         callback_object.append(i.split()[2])
#     #print(callback_object)
    
#     #callback_object -> duration
#     S = l.index('Callback instances:\n')
#     E = l.index('Lifecycle state machines:\n')
#     for i in callback_object:
#         d= [line for line in l[S:E] if i in line]
#     k=0
#     for i in d:
#         if k < a:
#             print(i)
#             k+=1

# for i in pairs:
#     print("request latency")
#     req_pr(i.split(':')[0].upper().replace('X', 'x'))
#     print("response latency")
#     res_pr(i.split(':')[1].upper().replace('X', 'x'))
#     print("service callback duration")
#     cb_pr(str(int(i.split(':')[0],16)))